# Pathfinder Master Notebook  
### (Author - Mark McDonald - 5/12/2019)
This notebook uses the following support scripts to preprocess data and train models:  
1) data_preprocess.py (imported as pf_preprocess)  
2) video_support_processes.py (imported as pf_video)  
3) models.py (imported as pf_models)



## STEP 1 - Set Backend For GPU Support
This can be commented out if an Nvidia GPU is being used

In [25]:
import os
# fix libiomp5.dylib error for mac
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [26]:
# OPTIONAL SETTING
# Here we override the keras backend env variable to use plaidml
# plaidml can make use of AMD GPUs 
# This assignment needs to be added before loading keras libraries

# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# to install plaidML, activate appropriate environment and then:
#   pip install -U plaidml-keras
#   plaidml-setup

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import keras
keras.__version__

'2.2.4'

## STEP 2 - Check that you are in the correct current directory

In [28]:
# Check that the current directory is '.../code'
pwd = !pwd
if pwd[0][-4:] != "code":
    print("ERROR: You're currently not in the project's code directory.")
    print(pwd[0])
else:
    print("SUCCESS: You are in the 'code' directory")
    print(pwd[0])
    import data_preprocess as pf_preprocess
    import models as pf_model
    import video_support_processes as pf_video

SUCCESS: You are in the 'code' directory
/Users/markmcdonald/Desktop/pathfinder/code


## STEP 3 - Set Variables
These must be set to match the local environment.  
No other items in the notebook need to be modified except these varables.

In [29]:
# This is where data downloaded from http://www.mikeprocopio.com/labeledlagrdata.html should exist
original_train_dir = '/Volumes/Photos-BACKUP/89FinalProject/89FinalData/orig_train_data'
original_test_dir = '/Volumes/Photos-BACKUP/89FinalProject/89FinalData/orig_test_data'

# This is where directories containing data extracted from original data will be placed.
base_dir = '/Volumes/Photos-BACKUP/89FinalProject/89FinalData'

# This is the percentage of validation data
val_split = .2

In [30]:
# Test that supplied source data directory exists
if not os.path.exists(original_train_dir):
    print("ERROR: Please supply a valid local directory that hold downloaded source train data.")
    print("Source data can be downloaded from http://www.mikeprocopio.com/labeledlagrdata.html.")
else:
    print("SUCCESS: Found original train data directory: {}".format(original_train_dir))

# Test that supplied test data directory exists
if not os.path.exists(original_test_dir):
    print("ERROR: Please supply a valid local directory that hold downloaded source test data.")
    print("Source data can be downloaded from http://www.mikeprocopio.com/labeledlagrdata.html.")
else:
    print("SUCCESS: Found original test data directory: {}".format(original_test_dir))
    

# Test that supplied base data directory exists
if not os.path.exists(base_dir):
    print("ERROR: Please create a directory that will be used to store extracted source data.")
    print("ERROR: Doesn't Exist: ", base_dir)
else:
    print("SUCCESS: Found base directory: {}".format(base_dir))
    

SUCCESS: Found original train data directory: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/orig_train_data
SUCCESS: Found original test data directory: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/orig_test_data
SUCCESS: Found base directory: /Volumes/Photos-BACKUP/89FinalProject/89FinalData


### Create Train and Validation Directories
Create necessary support directories under the 'base_dir'

In [31]:
# Here we establish the supporting directories for our training and validation data
# The True parameter will erase any pre-existing data in the directries
# Setting this parameter to False will only set the variables and assumed that directories already exist with data
train_dir, train_video_dir, val_dir, test_dir, results_dir = pf_preprocess.set_directories(base_dir, False)

Train Directory already existed and was retained: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/train
Train Images directory already exists: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/train/images
Train Images data directory already exists: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/train/images/data
Train Masks directory already exists: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/train/masks
Train Masks data directory already exists: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/train/masks/data
Video Dir already exists: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/train/video
Validation Directory already existed and was retained: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/val
Validation Images Dir already exists: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/val/images
Validation Data Images Dir already exists: /Volumes/Photos-BACKUP/89FinalProject/89FinalData/val/images/data
Validation Masks Dir already exists: /Volumes/Photos-BACKUP/89Fin

## STEP 4 -  Extract Data From Downloaded Files
Use the downloaded MAT files to extract the image and mask data saving the images and masks to supporting directories

In [ ]:
# Here we extract the image and mask data from our original data stored in MAT files
pf_preprocess.create_img_and_mask_data(original_train_dir, train_dir)

# For the test directories, masks won't be created
pf_preprocess.create_img_and_mask_data(original_test_dir, test_dir)

In [ ]:
# Validation images and masks are pulled out of the training set and put into validation directories
pf_preprocess.create_val_set(train_dir, val_dir, val_split)

We now have images and masks in separate directories and files.  
These will be used to train our network.

## SIDEBAR: Understanding the data set

Let's take a moment to understand the data that we are working with.  You can skip this, if you like, and proceed to step 5.  

The downloaded data is in MAT file format.  The MAT files contain a dictionary or data elements, some of which we will use for our process.

In [ ]:
# Review the original data
import scipy.io
mat_file_path = os.path.join(original_train_dir, 'labeled_lagr_data_640x480_DS1A/frame001.mat')
mat_file = scipy.io.loadmat(mat_file_path)
# Dictionary entries in the MAT file
for k in mat_file.keys():
    print(k)

Each MAT file has a number of pieces of data.  For this project, we will only use  
the 'im_rgb' and 'manual_human_labeling_mask'.

In [ ]:
# Original images are 640 x 480 (note that this is a vertical format)
orig_image = mat_file['im_rgb']
print("Image Shape: ", orig_image.shape)

In [ ]:
# Masks are the same size as the images but only one channel.
orig_mask = mat_file['manual_human_labeling_mask']
print("Mask Shape: ",orig_mask.shape)

In [ ]:
# The values in the mask range from 0 to 2 as expected
import numpy as np
print(np.amin(orig_mask))
print(np.amax(orig_mask))

0's are the ground plane.  1's are obstacles and 2's are unidentified.   
We are only interesting in the ground plane.  
We will modify these masks later so that they contain only 0's and 1's.  
First, we are going to create supoerimposed videos of the image sequences  
with the 3-value masks. This will make for a more interesting presented overlay.

In [ ]:
# Test file does not have the same data elements
mat_test_file_path = os.path.join(original_test_dir, 'labeled_lagr_data_640x480_DS2A_SupplementalFrames_101-624/frame620.mat')
mat_test_file = scipy.io.loadmat(mat_test_file_path)
# Dictionary entries in the MAT file
for k in mat_test_file.keys():
    print(k)

Note that the test files are missing the labeled mask.

## End of the SIDEBAR
Let's get back to work

## STEP 5 - Create a superimpsed video file based on original data
Use the images and masks to create a video with the masks superimposed on the images.  These videos are a simple demonstration to show what a superimposed mask looks like from the training data set.  These will be perfect masks because they were manually created.  Later, we will develop a model that will create the masks for us.

In [ ]:
pf_video.create_superimposed_video_from_MATFiles(original_train_dir, train_video_dir)

We now have a video that has an overlay of the path for each of the training scenes.

## STEP 6 - Change the mask data to Binary Values
The mask data contains 3 values that represent path(0), obstacle(1) and unknown(2).  
We are only interested in what is a path or not so using 3 values to train our model is not necessary  
and could potentially cause confusion during training and slow the training process with no benefit.  
We will change the masks to reflect the binary nature of our desired masks where the path is a 1 and everything else is a 0.  
This change is done in-place and will change the original masks.  
If we want 3-value masks again, we will need to recreate our dataset with the steps above.

In [ ]:
# In-place conversion of original masks to binary masks for training purposes
# This will destory the original masks and repalce them with binary masks
pf_preprocess.convert_jpg_mask_to_binary_mask(os.path.join(train_dir, 'masks'))
pf_preprocess.convert_jpg_mask_to_binary_mask(os.path.join(val_dir, 'masks'))

### Sample Image and Overlay
Take a sample to see that the conversion produces a reasonable mask

In [ ]:
# Displays and image with a mask overlay
from keras.preprocessing import image as kimage
def display_overlay(image, mask, ispath=True):
    if ispath:   
        image = kimage.load_img(image)
        mask = kimage.load_img(mask)
        image = np.uint8(image)
        mask = np.asarray(mask)

    noneType = type(None)
        
    if type(mask) != noneType:
        # make red layer for mask
        mask = np.moveaxis(mask, 2, 0)
        new_mask = np.zeros((3, mask.shape[1], mask.shape[2]))
        new_mask[0] = np.array(mask[0]==0)*180
        new_mask[1] = np.array(mask[0]==1)*180

        new_mask = np.moveaxis(new_mask, 0, 2)

        image = np.uint8(new_mask*.3 + image*.7)
            
    if image.shape[2] != 3:
        image = np.uint8((np.zeros(shape=(360,480,3))) + image)
    plt.imshow(np.uint8(image))


In [ ]:
imagefiles = sorted(os.listdir(os.path.join(train_dir, 'images', 'data')))
maskfiles = sorted(os.listdir(os.path.join(train_dir, 'masks', 'data')))
test_image = os.path.join(train_dir, 'images', 'data', imagefiles[0])
test_mask = os.path.join(train_dir, 'masks', 'data', maskfiles[0])

display_overlay(test_image, test_mask, ispath=True)

## Data Preparation is Complete
We now have a local data set of images and masks separated by training and validation sets.
We can now proceed to train our model.
***

## STEP 7 -  Model Training
Using get_models() will retrieve a series of defined models that can be trained.  
get_models() will return a list of all defined models and a description of the model.  
Each list entry consists of a tuple where the first element is a dictionary of paramters  
and the second element is the corresponding model.

### Data Generators (train, validate and test)

Generators are defined in the models.py script.  
One generator is defined for the training images and masks 
and another is defined for the validation images and masks.  
Each generator will return an image and a mask for the image.  
For the training generator, an augmented image and identically augmented mask is produced.
Batch sizes are defined in the 'description' dictionary from the model_tuple.

In [ ]:
from keras import models
from keras import layers

In [ ]:
# get a model tuple to test the generators with
model_tuples = []
model_tuples.append(pf_model.get_autoenc_model2(epochs=40, 
                                                 rot=20, 
                                                 zoom=[.7,1.0], 
                                                 hflip=True, 
                                                 vflip=True,
                                                 notes="test model"))

We use a support function to create a model.  The function will return a model as well as a 'description' dictionary that includes the settings used in the model.  This description is used during training to ensure that properly configured data generators are created and also serves as a way to see the model settings after it is done training and saved.

In [ ]:
model_tuples[0]

### Generator for training images and masks

In [ ]:
description = model_tuples[0][0]
train_generator = pf_model.get_img_mask_generator(os.path.join(train_dir, 'images'), 
                                             os.path.join(train_dir, 'masks'), 
                                             description)

In [ ]:
import cv2
image, mask = next(train_generator)

image1 = np.uint8(np.asarray(image[0]*255)) # genereated image is scaled to 1/255
mask1 = np.uint8(mask[0]) # mask wasn't rescaled because values are 0 or 1
display_overlay(image1,mask1, ispath=False)

The image produced is an augmented image where both the image and mask are identically augmented.

### Generator for validation images and masks

In [ ]:
val_generator = pf_model.get_img_mask_generator(os.path.join(val_dir, 'images'), 
                                           os.path.join(val_dir, 'masks'), 
                                           description,
                                           augment=False)

In [ ]:
import cv2
image, mask = next(val_generator)

image = np.uint8(np.asarray(image[0]*255)) # genereated image is scaled to 1/255
mask = np.uint8(mask[0]) # mask wasn't rescaled because values are 0 or 1
display_overlay(image,None, ispath=False)

In [ ]:
display_overlay(image,mask, ispath=False)

A radomnly selected image and mask from the generator are overlayed.  
We can see that the overlay matches the same augmentation as the image.

### Generator for test images
We will also use a generator to generate images from our test directory.  These images will be used to create masks from our model later.  This generator is created as a simplified way to generate test images in the same way that our model was trained with.

In [ ]:
test_generator = pf_video.get_test_img_generator(os.path.join(test_dir, 'images'), description, batch_size=1)

In [ ]:
import cv2
image = next(test_generator)
display_overlay(image[0], None, ispath=False)

Now that we are certain that our data generators are doing what we want, we can start training a real model.

### Train

In [32]:
# After training, we will save the description and the trained model weights in a pickle variable
# Here we define functions to help is with pickle file handling.
import pickle

# Use pickle to save history for later use
def pickle_save(variable, save_path):
    save_file = save_path + '/data.pickle'
    if os.path.isdir(save_path) == 0:
        os.mkdir(save_path)
    with open(save_file, 'wb') as f:
        pickle.dump(variable, f)

# Loading a prevously saved pickle file
def pickle_load(pickel_filepath):
    with open(pickel_filepath, 'rb') as f:
        data = pickle.load(f)
        return data

First, we need to use our get_model() function to get a model with provided argument settings.

In [33]:
# Get models for training
model_tuples = []
# The following model parameters were determined to provide the best overal results
# model_tuples.append(pf_model.get_autoenc_model(epochs=40, 
#                                                  rot=30, 
#                                                  zoom=[.6,1.0], 
#                                                  hflip=True, 
#                                                  vflip=True,
#                                                  dropout=.4,
#                                                  notes="rot30-.6-1.0zoom_dropout=.4BEST"))  
model_tuples.append(pf_model.get_autoenc_model3(epochs=30, 
                                                input_height=240,
                                                input_width=360,
                                                op_height=240,
                                                op_width=360,
                                                rot=30, 
                                                zoom=[.6,1.0], 
                                                hflip=True, 
                                                vflip=True,
                                                dropout=.2,
                                                notes="32-64-128-3-4-5"))  

In [36]:
# model_tuples[0][1].summary()

Next, is the training step.  This will train each model in the model_tuples list sequentially.  
This step will automatically create a test video with the trained model.  
Training progress charts can be seen in Tensorboard.

In [37]:
from tensorflow.keras.callbacks import TensorBoard
import math
import shutil

num_models = len(model_tuples)

# a model_tupled object can contain multiple models
# us pf_model.get_autoencoder_model() method to create tuples
for i, model_tuple in enumerate(model_tuples):
    description = model_tuple[0]
    model = model_tuple[1]
    
    # Compile model
    optimizer = description['optimizer']
    loss = description['loss']
    model.compile(optimizer=optimizer, loss=loss)
    
    # Add Tensorboard capability
    name = description["name"]
    logdir = os.path.join(base_dir, "logs" , name)
    tensorboard = TensorBoard(log_dir=logdir)

    # remove the logging directory for the model if it exists
    if os.path.isdir(logdir):  shutil.rmtree(logdir)
    
    # get number of images to train and validate
    num_timages = len(os.listdir(os.path.join(train_dir, 'images', 'data')))
    num_vimages = len(os.listdir(os.path.join(val_dir, 'images', 'data')))
    
    # create generators based on the model description variables
    train_gen = pf_model.get_img_mask_generator(os.path.join(train_dir, 'images'), 
                                             os.path.join(train_dir, 'masks'), 
                                             description,
                                             augment=True)
    val_gen = pf_model.get_img_mask_generator(os.path.join(val_dir, 'images'), 
                                         os.path.join(val_dir, 'masks'), 
                                         description,
                                         augment=False)
    
    epochs = description["epochs"]
    batch_size = description["batch_size"]
    
    print(model.summary())
    
    # step_per_epcoch and val_steps is numbner of images / batchsize
    history = model.fit_generator(train_gen, 
                                  steps_per_epoch=math.ceil(num_timages/batch_size), 
                                  epochs=epochs, verbose=1, 
                                  callbacks=[tensorboard],
                                  validation_data=val_gen, 
                                  validation_steps=math.ceil(num_vimages/batch_size),
                                  use_multiprocessing=True, 
                                  shuffle=True)
        
    results = model.evaluate_generator(val_gen, steps=math.ceil(num_vimages/batch_size))
    
    description.update({"history": history})
    description.update({"results": results})
    
    acc = str(np.round(results, decimals=4))
    print("Final Accuracy: ", acc)
    
    # Create overlay text and prepare a directory to save video and pickle file
    overlay_text = name + "_loss=" + acc
    source_dir = test_dir
    target_dir = os.path.join(results_dir, overlay_text) 
    os.mkdir(target_dir)
    
    # Save description and model in a pickle file
    pickle_save((description, model), target_dir)
    
    print(description)

    # Create video with mask overlay generated from model
    pf_video.create_video_with_test_data(source_dir, model, description, target_dir)

Creating generator:
	/Volumes/Photos-BACKUP/89FinalProject/89FinalData/train/images
	/Volumes/Photos-BACKUP/89FinalProject/89FinalData/train/masks
	Augemntation:  True
Found 480 images belonging to 1 classes.
Found 480 images belonging to 1 classes.
Creating generator:
	/Volumes/Photos-BACKUP/89FinalProject/89FinalData/val/images
	/Volumes/Photos-BACKUP/89FinalProject/89FinalData/val/masks
	Augemntation:  False
Found 120 images belonging to 1 classes.
Found 120 images belonging to 1 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 240, 360, 3)       0         
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 240, 360, 32)      896       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 80, 120, 32)       0         
_________________________________________

## Done!
A superimposed test video has been created in the directory indicated at the end of the output in the previous step.  The model parameters can be changed to explore settings that may provide better results.  Please let me know if you find a better combination of parameters.  mcdomx@me.com.

---  
---  

## SUPPORT PROCESSES: Variational AutoEncoder Design
These steps are to help with model design so that the dimensions can be seen.  The final model is stored in the models.py support script.

In [23]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout
from keras.models import Model
from keras import backend as K
from keras import regularizers

input_img = Input(shape=(240, 360, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((3, 3), padding='same')(x)
x = Dropout(.3)(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((4, 4), padding='same')(x)

# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = MaxPooling2D((3, 3), padding='same')(x)

# x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
# x = MaxPooling2D((3, 3), padding='same')(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((5, 5), padding='same')(x)

encoder = Model(input_img, encoded)

# at this point the representation is (15, 20, 64) i.e. 19200-dimensional
x = Conv2D(256, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 3))(x)

# x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
# x = UpSampling2D((3, 3))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((3, 3))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [24]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 240, 360, 3)       0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 240, 360, 32)      896       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 80, 120, 32)       0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 80, 120, 32)       0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 80, 120, 64)       18496     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 20, 30, 64)        0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 20, 30, 128)       73856     
__________

In [ ]:
import models as pf_model

In [ ]:
test_model = pf_model.get_autoenc_model2(epochs=40, 
                                                 rot=30, 
                                                 zoom=[.6,1.0], 
                                                 hflip=True, 
                                                 vflip=True,
                                                 dropout=.4,
                                                 notes="rot30-.6-1.0zoom_dropout=.4BEST")

In [ ]:
test_model[1].summary()